# Merging datasets

## Global imports and variables

In [9]:
# Import for interactive notebook (see:
# https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html)
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

# Import to list files in directories
import glob

# Import for regular expressions
import re

# Imports for path operations
import os
import os.path

# For date operations
from datetime import datetime

import pandas as pd
pd.set_option('max_colwidth', -1)
import matplotlib.pyplot as plt

import numpy as np
import configparser

# import jtplot module in notebook
from jupyterthemes import jtplot

# choose which theme to inherit plotting style from
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
jtplot.style(theme='onedork')


In [10]:
RESULTS_DIR = "/Users/gomerudo/workspace/thesis_results"

def rettext(text):
    return text

form_item_layout = Layout(
    width="50%"
)

## Selecting the desired results

In [11]:
import os.path
results_sorted = sorted(glob.glob("{dir}/*".format(dir=RESULTS_DIR)))
dict_widgets = {}
list_widgets = []
for result in results_sorted:
    hbox = widgets.Box()
    dict_widgets[result] = widgets.Checkbox(
        value=False,
        description=os.path.basename(result),
        disabled=False
    )
    list_widgets.append(dict_widgets[result])

half = len(results_sorted)//2

left_box = widgets.VBox(list_widgets[:half])
right_box = widgets.VBox(list_widgets[half:])
display(widgets.HBox([left_box, right_box]))

select = widgets.Dropdown(
    options=[('Chained', 1), ('Multi-branch', 2)],
    value=1,
    description='Type of results:',
)
display(widgets.HBox([select]))

In [11]:
# Run to reset the checkboxes
for key, widget in dict_widgets.items():
    widget.value = False

## Do the merge

In [12]:
# Get the selected cells
selected_dirs = []
for key, widget in dict_widgets.items():
    if widget.value:
        selected_dirs.append(key)
        
selected_dirs = sorted(selected_dirs)
selected_dirs

db_experiments_final = pd.DataFrame()

for directory in selected_dirs:
    db_src = sorted(
        glob.glob(
            "{root}/experiment*/db_experiments.csv".format(root=directory)
        )
    )[0]
    current_db = pd.read_csv(db_src)
    db_experiments_final = db_experiments_final.append(current_db, ignore_index=True)

db_experiments_final.shape

(59965, 9)

In [13]:
db_experiments_final = db_experiments_final.drop_duplicates(subset='dataset-nethash', keep='first')
db_experiments_final.shape

(33312, 9)

In [14]:
db_experiments_final.to_csv( "{root}/db_experiments.csv".format(root=RESULTS_DIR), index=False)